In [2]:
import pandas as pd
from openpyxl import load_workbook
from datetime import date, datetime, timedelta
from openpyxl.utils.dataframe import dataframe_to_rows

In [3]:
details_file_path = 'C:/Prajeen/My Docs/Equity/1.Equity Details/1.Fund Analysis/Fund Details/'
graphExcel = '2.Weekly fund Graph.xlsx'

In [4]:
def getDfFromXl(excel):
    headers = list(excel.values)[0]
    content = list(excel.values)[1:]
    return pd.DataFrame(data=content,columns=headers)

In [5]:
def getToday():
    return (date.today()).strftime('%d/%m/%Y')
    #return (date.today()- timedelta(1)).strftime('%d/%m/%Y')

In [6]:
evaData = load_workbook(details_file_path+graphExcel)
evaDf = getDfFromXl(evaData['evaluation'])
resultPage = evaData['result']
indEvaDf = evaDf.set_index(['Type', 'Date'])

In [7]:
for fType in evaDf.Type.unique():   
    slicedDf = indEvaDf.loc[fType,getToday()]
    retPoints = ['1W','1M','3M','6M']
    retData = slicedDf[retPoints]
#    print(fType)

    retMean = retData.mean()
    retRange = retData.max() - retData.min()
#    print(retMean)
#    print(retRange)

    for val in retPoints:
        newCol = val+'Norm'
        retData[newCol] = retData[val].apply(lambda x: (x-retMean[val])/retRange[val])

    '''# Other choices of multipliers are 
       # 0.2, 0.6, 0.9, 1
       # 0.25, 0.6, 0.9, 1
       # 0.25, 0.5, 0.9, 1
    '''
    retData = retData.assign(normSum6M = lambda x: x['1WNorm']*0.25 + x['1MNorm']*0.7 + x['3MNorm']*0.9 + x['6MNorm'])
    retData = (retData.reset_index()).round(2)

    for line in dataframe_to_rows(retData):
        resultPage.append(line)
    
evaData.save(details_file_path+graphExcel)
print('execution finished!')

C:\Prajeen\_Workstation\Anaconda\lib\site-packages\ipykernel\__main__.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
C:\Prajeen\_Workstation\Anaconda\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


execution finished!
